# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [13]:
#pip install gmaps

In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from config import vactionPy_apiKey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
csv_path = "data/weather_df.csv"

weather_df = pd.read_csv(csv_path, encoding="utf-8")
weather_df

,City,Country,Lat,Lng,Temperature,Humidity,Cloudiness,Wind (MPH)
0,Punta Arenas,CL,-53.1500,-70.9167,48.092,81,75,8.23
1,Ayan,RU,56.4500,138.1667,26.978,87,100,5.98
2,Castro,BR,-24.7911,-50.0119,59.000,85,54,5.58
3,Bluff,NZ,-46.6000,168.3333,62.006,71,18,2.68
4,Kapaa,US,22.0752,-159.3190,69.278,68,75,8.75
...,...,...,...,...,...,...,...,...
564,Mildura,AU,-34.2000,142.1500,84.578,17,55,3.09
565,Bykovo,RU,55.6361,38.0803,37.580,100,90,3.00
566,Novi Pazar,RS,43.1367,20.5122,26.762,92,100,1.80
567,Jadu,LY,31.9530,12.0261,51.800,84,43,7.44


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
# Configure gmaps with API key
gmaps.configure(api_key=vactionPy_apiKey)

In [17]:
# Store 'Latitude' and 'Longitude' as locations
locations = weather_df[["Lat", "Lng"]].astype(float)

# 'Humidity' as float data type
humidity = weather_df["Humidity"].astype(float)

In [18]:
# Create a Humidity Heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
hotel_df = weather_df.loc[(weather_df['Temperature']>= 65)
                           & (weather_df['Temperature']< 80)
                            & (weather_df['Humidity']>= 10)
                           & (weather_df['Humidity']< 20)]
hotel_df.dropna(how='any')

hotel_df

,City,Country,Lat,Lng,Temperature,Humidity,Cloudiness,Wind (MPH)
133,Saint George,US,37.1041,-113.5841,65.264,12,1,1.54
248,Yilan,TW,24.7570,121.7530,71.672,19,100,2.20
440,Aswān,EG,24.0934,32.9070,77.972,11,2,3.36
470,Sahuaripa,MX,29.0500,-109.2333,71.888,12,13,2.27


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
# Google Places - hotel finder

hotel_list = []

for index, city in hotel_df.iterrows():
    #print(city[1], city[2])
    lat = city[2]
    lng = city[3]
    target_search = "lodging"
    params = {
    "location": f"{lat},{lng}",
    "types": target_search,
    "rankby": "distance",
    "key": vactionPy_apiKey
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params)
    hotel_response = response.json()
    hotel_name = hotel_response["results"][0]["name"]
    hotel_list.append(hotel_name)

# create new column 'Hotel Name' and fill with generated hotels

hotel_df['Hotel Name'] = hotel_list

<ipython-input-21-60c1dc3921d1>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = hotel_list


In [22]:
# display Hotel data frame
hotel_df

,City,Country,Lat,Lng,Temperature,Humidity,Cloudiness,Wind (MPH),Hotel Name
133,Saint George,US,37.1041,-113.5841,65.264,12,1,1.54,Kelsch Metal Roofing
248,Yilan,TW,24.7570,121.7530,71.672,19,100,2.20,賓成精品商旅
440,Aswān,EG,24.0934,32.9070,77.972,11,2,3.36,Sakn Aswan
470,Sahuaripa,MX,29.0500,-109.2333,71.888,12,13,2.27,Hotel El Molino


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [25]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations,
                                 info_box_content = hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))